In [83]:
from copy import deepcopy

file = open("../Data/input_03.txt", "r")
content = file.read().split('\n')[:-1]
file.close()

# symbols = {}

# for el in list(''.join(content)):
#     symbols[el] = 1

# print(symbols.keys())

# relavent symbols
search_symbols = ['*', '=', '/', '%', '#', '&', '$', '-', '@', '+']
numbers = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']


# create a symbol mask from map  [search symbol] -> 1  [. 0-9] -> 0
symbol_mask = []
# create a number mask from map  [0-9] -> 1  [otherwise] -> 0
number_mask = []

for line in content:
    symbol_mask_line = []
    number_mask_line = []

    for c in list(line):

        if c in search_symbols:
            symbol_mask_line.append(1)
            number_mask_line.append(0)

        elif c in numbers:
            symbol_mask_line.append(0)
            number_mask_line.append(1)

        else:
            symbol_mask_line.append(0)
            number_mask_line.append(0)

    symbol_mask.append(symbol_mask_line)
    number_mask.append(number_mask_line)
    

def mask_expansion(mask, i, j, last_index):

    mask[i][j] = 1

    # not first element
    if j != 0:
        mask[i][j-1] = 1

    # not last element
    if j != last_index:
        mask[i][j+1] = 1

    return mask


# expand the symbol mask
symbol_mask_exp = deepcopy(symbol_mask)

for i, line_mask in enumerate(symbol_mask):
    
    for j, el in enumerate(line_mask):
        
        last_index = len(line_mask)-1

        if el == 1:
            
            symbol_mask_exp = mask_expansion(symbol_mask_exp, i, j, last_index)

            # not first line
            if i != 0:
                symbol_mask_exp = mask_expansion(symbol_mask_exp, i-1, j, last_index)

            # not last line
            if i != 0:
                symbol_mask_exp = mask_expansion(symbol_mask_exp, i+1, j, last_index)


# Go through a make an index mapping of number 
# [. . . 9 9 8 . . 3 3 . . 8 3 1 . .]  >>  [. . . 1 1 1 . . 2 2 . . 3 3 3 . .]
number_idx = deepcopy(number_mask)
counter = 1
number_dict = {}

for i, number_line in enumerate(number_mask):

    element = 0
    last_index = len(number_line)-1
    
    number = []

    while element < last_index+1:
        
        
        if number_line[element]==1:
        
            number_idx[i][element] = counter
            number.append(content[i][element])

            if (element == last_index):
                number_dict[counter] = int(''.join(number))
                number = []
                counter += 1

            elif number_idx[i][element+1] == 0:
                number_dict[counter] = int(''.join(number))
                number = []
                counter += 1

        element += 1


# Finding indices of number that are adjacent to a symbol
connected = []
for i, line_mask in enumerate(symbol_mask_exp):
    
    for j, el in enumerate(line_mask):
        if el == 1:
            connected.append(number_idx[i][j])

# set to remove duplicates of indices and remove 0
connected_indices = list(set(connected).symmetric_difference([0]))   

# Find sum of all numbers from number_dict (that connects index to numbers)
sum = 0

for ci in connected_indices:
   sum += number_dict[ci]


In [130]:
sum

557705